# TP4 : Modèle de Markov cachés

* Nous allons construire un modèle de Markov caché (dorenavant MMC) de séries historiques financières. Pour cela, vous allez récupérer les séries des rendements journaliers discrétisés en
{−1, 0, +1} obtenues lors du TD n◦1

In [12]:
import pandas as pd

CAC40 = pd.read_csv('CAC40.csv')
CAC40 = CAC40.loc[:,['Close']]
CAC40.insert(0,'Jour',[i for i in range(1, len(CAC40)+1)])
print(CAC40.head(3))

DAX = pd.read_csv('DAX30.csv')
DAX = DAX.loc[:,['Close']]
DAX.insert(0,'Jour',[i for i in range(1, len(DAX)+1)])
print(DAX.head(3))

FTSE = pd.read_csv('FTSE.csv', sep=";")
print(FTSE.head(3))

DJI = pd.read_csv('DJI.csv', sep=";")
print(DJI.head(3))

IXIC = pd.read_csv('NASDAQ.csv')
IXIC = IXIC.loc[:,['Close']]
IXIC.insert(0,'Jour',[i for i in range(1, len(IXIC)+1)])

print(IXIC.head(3))


import numpy as np

def score_function(DATAFRAME):
    score = [0]
    for i in range(1,len(DATAFRAME)):
        taux = np.log(DATAFRAME['Close'][i]/DATAFRAME['Close'][i-1])
        if taux > 0.005:
            score.append(1)
        if taux <= 0.005 and taux >=-0.005 :
            score.append(0)
        if taux < -0.005:
            score.append(-1)
    DATAFRAME.insert(2,'Score', score)


   Jour        Close
0     1  5590.790039
1     2  5559.569824
2     3  5472.359863
   Jour         Close
0     1  13906.669922
1     2  13873.969727
2     3  13643.950195
   Jour    Close
0     1  6715.42
1     2  6695.07
2     3  6638.85
   Jour     Close
0     1  31176.01
1     2  30996.98
2     3  30960.00
   Jour         Close
0     1  13530.910156
1     2  13543.059570
2     3  13635.990234


In [13]:
score_function(DAX)
score_function(CAC40)
score_function(IXIC)
score_function(DJI)
score_function(FTSE)

print( "---- DAX ----- \n",DAX.head(3))
print("---- CAC40 ----- \n",CAC40.head(3))
print("---- IXIC ----- \n",IXIC.head(3))
print("---- DJI ----- \n",DJI.head(3))
print("---- FTSE ----- \n",FTSE.head(3))

---- DAX ----- 
    Jour         Close  Score
0     1  13906.669922      0
1     2  13873.969727      0
2     3  13643.950195     -1
---- CAC40 ----- 
    Jour        Close  Score
0     1  5590.790039      0
1     2  5559.569824     -1
2     3  5472.359863     -1
---- IXIC ----- 
    Jour         Close  Score
0     1  13530.910156      0
1     2  13543.059570      0
2     3  13635.990234      1
---- DJI ----- 
    Jour     Close  Score
0     1  31176.01      0
1     2  30996.98     -1
2     3  30960.00      0
---- FTSE ----- 
    Jour    Close  Score
0     1  6715.42      0
1     2  6695.07      0
2     3  6638.85     -1


## 1. Construisez avec Pomegranate le MMC décrit ci-dessus. 
* Pour commencer, vous mettrez des probabilités arbitraires. 
Attention : pour quelque raison mistérieuse, Pomegranate ajoute toujours un état initial start et un état final end même si vous dites expressément au constructeur de la classe HiddenMarkovModel que vous ne les voulez pas. Il suffit alors d’ajouter une transition de probabilité 1/2 de start à bullish ainsi que de start à bearish

In [14]:
from pomegranate import *
from pomegranate import HiddenMarkovModel

A chaque instant **t**, **Xt** représente l'état caché, et **Yt** représente une observation à cet instant
* soit **Xt** prend les valeurs dans le domaine **['state hausse', 'state baisse']** , c'est à dire bullish ou bearish au jour t
* soit **Yt** prend les valeurs dans le domaine **[-1,0,1]**, c'est à dire c'est à dire, court en baisse, neutre, ou hausse au jour t

* Le modèle qu’on construira ne pourrait être plus simple : on fera l’hypothèse qu’il n’existent que deux tendances : à la hausse (bullish) et à la baisse (bearish. Ces deux tendances sont les deux états “cachés” (non observables directement) de notre MMC ; 
* Les variations quotidiennes sont les observations “émises” par chaque tendance, suivant sa propre distribution de probabilité.

* On met des probabilité arbitaire au début : 

In [243]:
dist_hausse = DiscreteDistribution({'-1': 0.1 ,'0': 0.2 ,"1":0.7}) #bullish
dist_baisse = DiscreteDistribution({'-1': 0.6 ,'0': 0.3,"1":0.1}) #bearish

s_hausse = State( dist_hausse, name="state hausse" ) #bullish
s_baisse = State( dist_baisse, name="state baisse" ) #bearish

model = HiddenMarkovModel('first test')
model.add_states(s_hausse, s_baisse)
model.add_transition( model.start, s_hausse, 0.5 )
model.add_transition( model.start, s_baisse, 0.5 )

model.add_transition( s_baisse, s_hausse, 0.2 ) #bearish -> bullish
model.add_transition( s_baisse, s_baisse, 0.8 ) #bullish -> bullish

model.add_transition( s_hausse, s_baisse, 0.3 ) #bullish -> bearish
model.add_transition( s_hausse, s_hausse, 0.7 ) #bearish -> bearish

model.bake()

In [244]:
[s.name for s in model.states]

['state baisse', 'state hausse', 'first test-start', 'first test-end']

In [246]:
column_order = ["state baisse","state hausse","first test-start",'first test-end']  # Override the Pomegranate default order
column_names = [s.name for s in model.states]
order_index = [column_names.index(c) for c in column_order]# re-order the rows/columns to match the specified column order
transitions = model.dense_transition_matrix()[:, order_index][order_index, :]
print("La matrice de transition, P(Xt|Xt-1):")
print(transitions)
print("\n La probabilité de transition de Bullish à Bearish est de {:.0f}%".format(100 * transitions[1, 0]))

La matrice de transition, P(Xt|Xt-1):
[[0.8 0.2 0.  0. ]
 [0.3 0.7 0.  0. ]
 [0.5 0.5 0.  0. ]
 [0.  0.  0.  0. ]]

 La probabilité de transition de Bullish à Bearish est de 30%


## 2. Créez, avec un petit script Python, une série historique de test, où vous allez simuler un marché qui alterne, sur un total di 1000 jours, 50 jours de taureau et 50 jours d’ours ; 
* lorsque le marché est bullish,
* P (st= −1) = 0.2, P (st= 0) = 0.3, P (st= +1) = 0.5,
* et lorsqu’il est bearish,
* P (st= −1) = 0.5, P (st= 0) = 0.2, P (st= +1) = 0.3.



In [247]:
print("          ",-1,"     ", 0,"    ", 1) 
print("bullish = ",[0.2*50, 0.3*50, 0.5*50]) #10 fois -1, 15 fois 0, 25 fois 1
print("bearish = ",[0.5*50, 0.2*50, 0.3*50]) #25 fois -1 ,10 fois 0, 15 fois 1

           -1       0      1
bullish =  [10.0, 15.0, 25.0]
bearish =  [25.0, 10.0, 15.0]


In [248]:
import random

bullish = [] 
for x in range(0, 10):
    bullish.append(-1) 

for x in range(0, 15):
    bullish.append(0) 

for x in range(0, 25):
    bullish.append(1) 

random.shuffle(bullish)


bearish = [] 
for x in range(0, 25):
    bearish.append(-1) 

for x in range(0, 10):
    bearish.append(0) 

for x in range(0, 15):
    bearish.append(1) 

random.shuffle(bearish)

print(bullish)
ps = pd.Series(i for i in bullish)
counts = ps.value_counts()
print(counts)

print(bearish)
ps2 = pd.Series(i for i in bearish)
counts2 = ps2.value_counts()
print(counts2)

[-1, 1, 1, 1, 0, 1, -1, -1, 0, 1, 1, 1, 0, -1, 0, 1, 1, 1, 1, 1, 1, -1, 0, 0, 1, -1, 1, 0, 1, -1, 1, -1, 1, 0, 0, 0, 0, 1, 1, 1, -1, 0, 1, 0, 1, 0, 1, -1, 1, 0]
 1    25
 0    15
-1    10
dtype: int64
[0, 0, 1, -1, 0, 1, -1, 0, -1, -1, 1, 1, -1, 1, -1, 0, -1, 1, -1, 1, -1, 1, -1, -1, 1, -1, 0, 1, -1, -1, -1, -1, 1, 1, 0, -1, -1, 0, -1, -1, 0, -1, 1, 1, -1, 0, 1, -1, -1, -1]
-1    25
 1    15
 0    10
dtype: int64


In [249]:
test_serie = []
state_serie = []
for i in range(0,10):
    for num in bullish: 
        test_serie.append(str(num))
        state_serie.append('state hausse')
    for num in bearish: 
        test_serie.append(str(num))
        state_serie.append('state baisse')


print(len(test_serie) )


1000


### 3. Entraînez le MMC (méthode fit) avec cette série de test. 
* Une fois entraîné, le MMC devrait reconnaître (méthode viterbi) les deux tendances, au fur et à mesure qu’elles s’alternent
* Vérifiez aussi que les probabilités d’émission des deux états reflètent bien les paramètres que vous avez utilisé pour produire la série de test

### Valeur par jour et Etat réel par jour de ma série 

In [250]:
print(test_serie[0:60])
print(state_serie[0:60])

['-1', '1', '1', '1', '0', '1', '-1', '-1', '0', '1', '1', '1', '0', '-1', '0', '1', '1', '1', '1', '1', '1', '-1', '0', '0', '1', '-1', '1', '0', '1', '-1', '1', '-1', '1', '0', '0', '0', '0', '1', '1', '1', '-1', '0', '1', '0', '1', '0', '1', '-1', '1', '0', '0', '0', '1', '-1', '0', '1', '-1', '0', '-1', '-1']
['state hausse', 'state hausse', 'state hausse', 'state hausse', 'state hausse', 'state hausse', 'state hausse', 'state hausse', 'state hausse', 'state hausse', 'state hausse', 'state hausse', 'state hausse', 'state hausse', 'state hausse', 'state hausse', 'state hausse', 'state hausse', 'state hausse', 'state hausse', 'state hausse', 'state hausse', 'state hausse', 'state hausse', 'state hausse', 'state hausse', 'state hausse', 'state hausse', 'state hausse', 'state hausse', 'state hausse', 'state hausse', 'state hausse', 'state hausse', 'state hausse', 'state hausse', 'state hausse', 'state hausse', 'state hausse', 'state hausse', 'state hausse', 'state hausse', 'state hauss

### Etat réel prédit par jour

In [ ]:
#print(", ".join(state.name for i, state in model.viterbi(test_serie)[1]))

In [251]:
import sklearn
positive = 0

for i, state in model.viterbi(test_serie)[1] :
    if state.name == state_serie[i]:
        positive +=1

taux_de_reussite = positive/len(test_serie)*100
print("Le taux de bonne prédiction pour les états de bullish ou bearish est de : ", taux_de_reussite,"%")


Le taux de bonne prédiction pour les états de bullish ou bearish est de :  37.0 %


* On lance une première prédiction arbitraire avec nos proba choisies au départ 

# fit
* On lance la méthode .fit() sur notre model à partir de l'ensemble de test généré auparavant 

In [ ]:
import numpy as np
test_s = [np.array( list(map(str, test_serie)) )]
model.fit(test_s, verbose=True)

* On relance la prédiction et on observe qu'on a de meilleurs résultats

In [271]:
import sklearn
positive = 0

for i, state in model.viterbi(test_serie)[1] :
    if state.name == state_serie[i]:
        positive +=1

taux_de_reussite = positive/len(test_serie)*100
print("Le taux de bonne prédiction est de : ", taux_de_reussite,"%")


Le taux de bonne prédiction est de :  49.0 %


* Pour rappel, la distribution choisies initialement (discrète) :
- ({'-1': 0.6 ,'0': 0.3,"1":0.1}) -> baisse = bearish
- ({'-1': 0.1 ,'0': 0.2 ,"1":0.7}) -> hausse = bullish

* On note que ces probabilité ont changé pour se rapprocher de celles recherchées : 
- ({'−1': 0.5, '0': 0.2, '1': 0.3}) -> baisse = bearish
- ({'−1': 0.2, '0': 0.3, '1': 0.5}) -> hausse = bullish


In [270]:
for s in model.states[0:2]:
    print(s.name)
    print(s.distribution.parameters)

state baisse
[{'-1': 0.43888891230357047, '0': 0.3271704475541853, '1': 0.23394064014224414}]
state hausse
[{'-1': 0.2553290688982378, '0': 0.1678097927608857, '1': 0.5768611383408764}]


* On a donc bien un algorithme qui se rapproche des valeurs souhaités, les deux états semblent bien représentés : 
- state baisse : 
    * -1 : 0.44 --> 0.5
    *  0 : 0.33 --> 0.2
    *  1 : 0.23 --> 0.3
- state hausse :
    * -1 : 0.26 --> 0.2
    *  0 : 0.17 --> 0.3
    *  1 : 0.58 --> 0.5

### Recherche plus en profondeur 


In [ ]:
import sklearn
positive = 0

for i, state in model.viterbi(test_serie)[1] :
    print(state.name, state_serie[i])
    if state.name == state_serie[i]:
        positive +=1

taux_de_reussite = positive/len(test_serie)*100
print("Le taux de bonne prédiction est de : ", taux_de_reussite,"%")

L'algorithme semble presque toujours alterner hausse et baisse chaque jour

* On voit que la matrice de transition a des valeurs différentes qu'auparavant et elle confirme cette hypothese
* Lorqu'on est en hausse on a de grande chance de baisser
* Lorqu'on est en baisse on a de grande chance de monter

(j'ai un peu du mal à interpréter ces résultats, car du coup on a une mauvaise représentation des périodes qui s'alternent, on a pas la notion des 50jours de bearish suivi des 50 de bullish, donc je me demande comment on pourrait faire pour ce model le prenne en compte. Car là, il alterne bearish et bullish mais chaque jour et pas tout les 50)

In [277]:
column_order = ["state baisse","state hausse","first test-start",'first test-end']  # Override the Pomegranate default order
column_names = [s.name for s in model.states]
order_index = [column_names.index(c) for c in column_order]# re-order the rows/columns to match the specified column order
transitions = model.dense_transition_matrix()[:, order_index][order_index, :]
print("La matrice de transition, P(Xt|Xt-1):")
print(transitions)
print("\n La probabilité de transition de Bullish à Bearish est de {:.0f}%".format(100 * transitions[1, 0]))
print("\n La probabilité de transition de Bearish à Bullish est de {:.0f}%".format(100 * transitions[0, 1]))

La matrice de transition, P(Xt|Xt-1):
[[0.09134949 0.90865051 0.         0.        ]
 [0.9663151  0.0336849  0.         0.        ]
 [1.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]]

 La probabilité de transition de Bullish à Bearish est de 97%

 La probabilité de transition de Bearish à Bullish est de 91%


### 4. Maintenant, appliquez le MMC aux séries réelles (celles que vous avez obtenues lors du TD n◦1). Cela veut dire entraîner le MMC avec une série, puis lui faire reconnaître les tendances sur la même série

In [278]:
print( "---- DAX ----- \n",DAX.head(3))
print("---- CAC40 ----- \n",CAC40.head(3))
print("---- IXIC ----- \n",IXIC.head(3))
print("---- DJI ----- \n",DJI.head(3))
print("---- FTSE ----- \n",FTSE.head(3))

---- DAX ----- 
    Jour         Close  Score
0     1  13906.669922      0
1     2  13873.969727      0
2     3  13643.950195     -1
---- CAC40 ----- 
    Jour        Close  Score
0     1  5590.790039      0
1     2  5559.569824     -1
2     3  5472.359863     -1
---- IXIC ----- 
    Jour         Close  Score
0     1  13530.910156      0
1     2  13543.059570      0
2     3  13635.990234      1
---- DJI ----- 
    Jour     Close  Score
0     1  31176.01      0
1     2  30996.98     -1
2     3  30960.00      0
---- FTSE ----- 
    Jour    Close  Score
0     1  6715.42      0
1     2  6695.07      0
2     3  6638.85     -1


#### Modele initial

In [315]:
dist_hausse = DiscreteDistribution({'-1': 0.2 ,'0': 0.3 ,"1":0.5}) #bullish
dist_baisse = DiscreteDistribution({'-1': 0.5 ,'0': 0.2,"1":0.3}) #bearish

s_hausse = State( dist_hausse, name="state hausse" ) #bullish
s_baisse = State( dist_baisse, name="state baisse" ) #bearish

model = HiddenMarkovModel('first test')
model.add_states(s_hausse, s_baisse)
model.add_transition( model.start, s_hausse, 0.5 )
model.add_transition( model.start, s_baisse, 0.5 )

model.add_transition( s_baisse, s_hausse, 0.2 ) #bearish -> bullish
model.add_transition( s_baisse, s_baisse, 0.8 ) #bullish -> bullish

model.add_transition( s_hausse, s_baisse, 0.3 ) #bullish -> bearish
model.add_transition( s_hausse, s_hausse, 0.7 ) #bearish -> bearish

model.bake()

In [297]:
test_s = [np.array( list(map(str, CAC40['Score'])) )]
CAC_model = model.fit(test_s)

In [298]:
for s in CAC_model.states[0:2]:
    print(s.name)
    print(s.distribution.parameters)

state baisse
[{'-1': 0.34056747828039446, '0': 0.28477837681391366, '1': 0.37465414490569193}]
state hausse
[{'-1': 0.13007047343449102, '0': 0.5924486818935579, '1': 0.27748084467195105}]


In [ ]:
test_s = [np.array( list(map(str, DAX['Score'])) )]
DAX_model = model.fit(test_s)

In [301]:
for s in DAX_model.states[0:2]:
    print(s.name)
    print(s.distribution.parameters)

state baisse
[{'-1': 0.41195308260134095, '0': 0.13248937674716443, '1': 0.4555575406514946}]
state hausse
[{'-1': 0.11050894519080467, '0': 0.7366248800590423, '1': 0.15286617475015318}]


In [307]:
test_s = [np.array( list(map(str, IXIC['Score'])) )]
IXIC_model = model.fit(test_s)

In [308]:
for s in IXIC_model.states[0:2]:
    print(s.name)
    print(s.distribution.parameters)

state baisse
[{'-1': 0.4396963767158781, '0': 0.1968494067422143, '1': 0.36345421654190757}]
state hausse
[{'-1': 0.2086438150033617, '0': 0.4549978233133221, '1': 0.33635836168331623}]


In [313]:
test_s = [np.array( list(map(str, DJI['Score'])) )]
DJI_model = model.fit(test_s)

In [314]:
for s in DJI_model.states[0:2]:
    print(s.name)
    print(s.distribution.parameters)

state baisse
[{'-1': 0.5448390417395277, '0': 1.0637298636417324e-07, '1': 0.45516085188748595}]
state hausse
[{'-1': 4.9942237827771015e-06, '0': 0.812603309236952, '1': 0.18739169653926524}]


In [ ]:
test_s = [np.array( list(map(str, FTSE['Score'])) )]
FTSE_model = model.fit(test_s)

In [317]:
for s in FTSE_model.states[0:2]:
    print(s.name)
    print(s.distribution.parameters)

state baisse
[{'-1': 0.2877825668770154, '0': 0.3352255970014446, '1': 0.37699183612154}]
state hausse
[{'-1': 8.62971542925609e-11, '0': 0.9389039992061563, '1': 0.061096000707546616}]
